In [213]:
# example of a wgan for generating handwritten digits
from numpy import expand_dims
from numpy import mean
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras import backend
from tensorflow.keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from keras.constraints import Constraint
from matplotlib import pyplot
import pandas as pd
import numpy as np

In [214]:
# clip model weights to a given hypercube
class ClipConstraint(Constraint):
	# set clip value when initialized
	def __init__(self, clip_value):
		self.clip_value = clip_value
 
	# clip model weights to hypercube
	def __call__(self, weights):
		return backend.clip(weights, -self.clip_value, self.clip_value)
 
	# get the config
	def get_config(self):
		return {'clip_value': self.clip_value}

In [215]:
# calculate wasserstein loss
def wasserstein_loss(y_true, y_pred):
	return backend.mean(y_true * y_pred)

In [216]:
def define_generator(G_in, dim=128, out_dim=2, lr=1e-3):
    x = Dense(dim)(G_in)
    x = Dense(dim*2, activation='relu')(x)
    x = Dense(dim*4, activation='relu')(x)
    G_out = Dense(out_dim)(x)
    G = Model(G_in, G_out)
    #opt = SGD(lr=lr)
    #G.compile(loss='binary_crossentropy', optimizer=opt)
    return G,G_out

In [241]:
# define the standalone critic model
def define_critic(D_in,dim=128,out_dim=1,lr=1e-3):
	const = ClipConstraint(0.01)
	x = Dense(dim * 4, activation='relu', kernel_constraint=const)(D_in)
	x = Dropout(0.1)(x)
	x = Dense(dim * 2, activation='relu', kernel_constraint=const)(x)
	x = Dropout(0.1)(x)
	x = Dense(dim, activation='relu', kernel_constraint=const)(x)
	D_out = Dense(out_dim)(x)
	D = Model(D_in, D_out)
	opt = RMSprop(lr=lr)
	D.compile(loss=wasserstein_loss, optimizer=opt)
	
	return D,D_out

In [242]:
# define the combined generator and critic model, for updating the generator
def define_gan(generator, critic,lr=1e-3):
	# make weights in the critic not trainable
	for layer in critic.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the critic
	model.add(critic)
	# compile model
	opt = RMSprop(lr=lr)
	model.compile(loss=wasserstein_loss, optimizer=opt)
	return model

In [84]:
'''# load images
def load_real_samples():
	df1 = pd.read_csv ("../Data/train.csv")
	X = np.array(df1)[:,1:]
	return X'''

'# load images\ndef load_real_samples():\n\tdf1 = pd.read_csv ("../Data/train.csv")\n\tX = np.array(df1)[:,1:]\n\treturn X'

In [243]:
df1 = pd.read_csv ("../Data/train.csv")
X = np.array(df1)[:,1:]
X= X[:,:2]

In [244]:
def generate_real_samples(dataset, n_samples):
	ix = randint(0, dataset.shape[0], n_samples)
	X = dataset[ix]
	y = -ones((n_samples, 1))
	return X, y

In [245]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [246]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels with 1.0 for 'fake'
	y = ones((n_samples, 1))
	return X, y

In [17]:
'''# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
	# prepare fake examples
	X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# plot images
	for i in range(10 * 10):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
	# save plot to file
	filename1 = 'generated_plot_%04d.png' % (step+1)
	pyplot.savefig(filename1)
	pyplot.close()
	# save the generator model
	filename2 = 'model_%04d.h5' % (step+1)
	g_model.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))'''

In [247]:
# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist):
	# plot history
	pyplot.plot(d1_hist, label='crit_real')
	pyplot.plot(d2_hist, label='crit_fake')
	pyplot.plot(g_hist, label='gen')
	pyplot.legend()
	pyplot.savefig('plot_line_plot_loss.png')
	pyplot.close()

In [248]:
# train the generator and critic
def train(g_model, c_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=5, n_critic=5):
	# calculate the number of batches per training epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# calculate the size of half a batch of samples
	half_batch = int(n_batch / 2)
	# lists for keeping track of loss
	c1_hist, c2_hist, g_hist = list(), list(), list()
	# manually enumerate epochs
	for i in range(n_steps):
		# update the critic more than the generator
		c1_tmp, c2_tmp = list(), list()
		for _ in range(n_critic):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update critic model weights
			c_loss1 = c_model.train_on_batch(X_real, y_real)
			c1_tmp.append(c_loss1)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update critic model weights
			c_loss2 = c_model.train_on_batch(X_fake, y_fake)
			c2_tmp.append(c_loss2)
		# store critic loss
		c1_hist.append(mean(c1_tmp))
		c2_hist.append(mean(c2_tmp))
		# prepare points in latent space as input for the generator
		X_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = -ones((n_batch, 1))
		# update the generator via the critic's error
		g_loss = gan_model.train_on_batch(X_gan, y_gan)
		g_hist.append(g_loss)
		# summarize loss on this batch
		print('>%d, c1=%.3f, c2=%.3f g=%.3f' % (i+1, c1_hist[-1], c2_hist[-1], g_loss))
		# evaluate the model performance every 'epoch'
		#if (i+1) % bat_per_epo == 0:
			#summarize_performance(i, g_model, latent_dim)
	# line plots of loss
	plot_history(c1_hist, c2_hist, g_hist)


In [211]:
# size of the latent space
latent_dim = 2
G_in = Input(shape=[2])
D_in = Input(shape=[2])
# create the critic
critic = define_critic(D_in,dim=128,out_dim=1,lr=1e-3):
# create the generator
generator = define_generator(Input(shape=[2]))
# create the gan
gan_model = define_gan(generator, critic)
# load image data
dataset = X
print(dataset.shape)


(745, 2)


In [212]:
train(generator, critic, gan_model, dataset, latent_dim)

TypeError: must be real number, not list